<a href="https://colab.research.google.com/github/AhmadHossain8/Food-Image-Classification/blob/main/2_model_Combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.optim.swa_utils import AveragedModel, SWALR
import numpy as np
import copy

from google.colab import drive
drive.mount('/content/drive')
import zipfile

DATA_SET_ZIP = '/content/drive/MyDrive/Colab Notebooks/Food/Dataset.zip'
with zipfile.ZipFile(DATA_SET_ZIP,"r") as zip_ref:
  zip_ref.extractall("data")

import os
import time
import pandas as pd
from PIL import Image
from torchvision import transforms,models
import matplotlib.pyplot as plt
from skimage import io,transform
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
path_of_training_data = '/content/data/training'
path_of_evaluation_data = '/content/data/evaluation'
path_of_validation_data = '/content/data/validation'

In [3]:
class Food_11_Dataset(Dataset):
  def __init__(self,data_dir, transform=None):
    self.date_dir = data_dir
    self.image_list = os.listdir(data_dir)
    self.transform = transform

  def __getitem__(self,idx):
    location = os.path.join(self.date_dir,self.image_list[idx])
    image = Image.open(location)
    if self.transform is not None:
            image = self.transform(image)
    label = self.image_list[idx]
    label = label.split("_")
    label = int(label[0])
    return image,label

  def __len__(self):
    return len(self.image_list)

In [4]:
LEARNING_RATE = 0.001
NUM_EPOCHS = 30
BATCH_SIZE = 10
NUM_CLASSES = 11

transform1 = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.50,0.50 ,0.50], std=[0.50, 0.50, 0.50]),
])

transform2 = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.50,0.50 ,0.50], std=[0.50, 0.50, 0.50]),
])

In [5]:
trainset1 = Food_11_Dataset(path_of_training_data,transform=transform1)
trainset_loader1 = torch.utils.data.DataLoader(trainset1, batch_size=BATCH_SIZE, shuffle=False)

evaluationset1 = Food_11_Dataset(path_of_evaluation_data,transform=transform1)
evaluationset_loader1 = torch.utils.data.DataLoader(evaluationset1, batch_size=BATCH_SIZE, shuffle=False)

validset1 = Food_11_Dataset(path_of_validation_data,transform=transform1)
validset_loader1 = torch.utils.data.DataLoader(validset1, batch_size=BATCH_SIZE, shuffle=False)


trainset2 = Food_11_Dataset(path_of_training_data,transform=transform2)
trainset_loader2 = torch.utils.data.DataLoader(trainset2, batch_size=BATCH_SIZE, shuffle=False)

evaluationset2 = Food_11_Dataset(path_of_evaluation_data,transform=transform2)
evaluationset_loader2 = torch.utils.data.DataLoader(evaluationset2, batch_size=BATCH_SIZE, shuffle=False)

validset2 = Food_11_Dataset(path_of_validation_data,transform=transform2)
validset_loader2 = torch.utils.data.DataLoader(validset2, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
class MyEnsemble(torch.nn.Module):
    def __init__(self, modelA,modelB):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        
    def forward(self,x1,x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x =   x1+x2
        x = torch.mul(x, (1.0/2.0))
        return torch.softmax(x, dim=1)


class MyEnsemble2(torch.nn.Module):
    def __init__(self, modelA,modelB):
        super(MyEnsemble2, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = torch.nn.Linear(22, 11)
        
    def forward(self, x1,x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1,x2), dim=1)
        x = self.classifier(x)
        return x

In [ ]:
PATH1 = "/content/drive/MyDrive/Test103/path1.pth"
PATH2 = "/content/drive/MyDrive/Test103/path2.pth"
PATH3 = "/content/drive/MyDrive/Test103/path3.pth"
PATH4 = "/content/drive/MyDrive/Test103/path4.pth"
PATH5 = "/content/drive/MyDrive/Test103/path5.pth"
PATH6 = "/content/drive/MyDrive/Test103/path6.pth"
PATH7 = "/content/drive/MyDrive/Test103/path7.pth"
PATH8 = "/content/drive/MyDrive/Test103/path8.pth"

PATH9 = "/content/drive/MyDrive/Test103/path9.pth"
PATH10 = "/content/drive/MyDrive/Test103/path10.pth"
PATH11 = "/content/drive/MyDrive/Test103/path11.pth"
PATH12 = "/content/drive/MyDrive/Test103/path12.pth"
PATH13 = "/content/drive/MyDrive/Test103/path13.pth"
PATH14 = "/content/drive/MyDrive/Test103/path14.pth"


#Traditional Ensemble
modelA = torch.hub.load('pytorch/vision:v0.13.0', 'alexnet', pretrained=True)
modelA.to(device)
modelB = torch.hub.load('pytorch/vision:v0.13.0', 'resnet50', pretrained=True)
modelB.to(device)
modelC = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
modelC.aux_logits = False 
modelC.to(device)
modelD = torch.hub.load('pytorch/vision:v0.13.0', 'vgg16', pretrained=True)
modelD.to(device)
modelA.classifier[6] = torch.nn.Linear(4096,11)
modelB.fc = torch.nn.Linear(2048, 11)
modelC.fc = torch.nn.Linear(2048, 11)
modelD.classifier[6] = torch.nn.Linear(4096,11)
model1 = MyEnsemble(modelA,modelB)
model1.to(device)
criterion1 = torch.nn.CrossEntropyLoss()
optimizer1 = torch.optim.SGD(model1.parameters(), lr=LEARNING_RATE)


if os.path.getsize(PATH1) > 0:
  checkpoint = torch.load(PATH1, map_location=torch.device(device))
  modelA.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH2) > 0:
  checkpoint = torch.load(PATH2, map_location=torch.device(device))
  modelC.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH3) > 0:
  checkpoint = torch.load(PATH3, map_location=torch.device(device))
  modelB.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH4) > 0:
  checkpoint = torch.load(PATH4, map_location=torch.device(device))
  modelD.load_state_dict(checkpoint['model_state_dict'])

epoch1 = 0
loss1 = 0.0
best_acc1 = 0.0
best_model_weights1 = []
if os.path.getsize(PATH9) > 0:
  checkpoint = torch.load(PATH9, map_location=torch.device(device))
  model1.load_state_dict(checkpoint['model_state_dict'])
  optimizer1.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch1 = checkpoint['epoch']
  loss1 = checkpoint['loss']
  best_acc1 = checkpoint['best_acc']
  best_model_weights1 = copy.deepcopy(model1.state_dict())


model1_info = {'train_loss' : [] , 'val_acc' : []}
PATH1_acc = "/content/drive/MyDrive/Test103/path1acc.pth"
if os.path.getsize(PATH1_acc) > 0:
  model1_info = torch.load(PATH1_acc, map_location=torch.device(device))


#Traditional Avg Ensemble
modelA1 = torch.hub.load('pytorch/vision:v0.13.0', 'alexnet', pretrained=True)
modelA1.to(device)
modelB1 = torch.hub.load('pytorch/vision:v0.13.0', 'resnet50', pretrained=True)
modelB1.to(device)
modelC1 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
modelC1.aux_logits = False 
modelC1.to(device)
modelD1 = torch.hub.load('pytorch/vision:v0.13.0', 'vgg16', pretrained=True)
modelD1.to(device)
modelA1.classifier[6] = torch.nn.Linear(4096,11)
modelB1.fc = torch.nn.Linear(2048, 11)
modelC1.fc = torch.nn.Linear(2048, 11)
modelD1.classifier[6] = torch.nn.Linear(4096,11)
model2 = MyEnsemble(modelA1,modelB1)
model2.to(device)
criterion2 = torch.nn.CrossEntropyLoss()
optimizer2 = torch.optim.SGD(model2.parameters(), lr=LEARNING_RATE)


if os.path.getsize(PATH5) > 0:
  checkpoint = torch.load(PATH5, map_location=torch.device(device))
  modelA1.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH6) > 0:
  checkpoint = torch.load(PATH6, map_location=torch.device(device))
  modelC1.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH7) > 0:
  checkpoint = torch.load(PATH7, map_location=torch.device(device))
  modelB1.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH8) > 0:
  checkpoint = torch.load(PATH8, map_location=torch.device(device))
  modelD1.load_state_dict(checkpoint['model_state_dict'])

epoch2 = 0
loss2 = 0.0
best_acc2 = 0.0
best_model_weights2 = []
if os.path.getsize(PATH10) > 0:
  checkpoint = torch.load(PATH10, map_location=torch.device(device))
  model2.load_state_dict(checkpoint['model_state_dict'])
  optimizer2.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch2 = checkpoint['epoch']
  loss2 = checkpoint['loss']
  best_acc2 = checkpoint['best_acc']
  best_model_weights2 = copy.deepcopy(model2.state_dict())


model2_info = {'train_loss' : [] , 'val_acc' : []}
PATH2_acc = "/content/drive/MyDrive/Test103/path2acc.pth"
if os.path.getsize(PATH2_acc) > 0:
  model2_info = torch.load(PATH2_acc, map_location=torch.device(device))




#Base Ensemble
modelA2 = torch.hub.load('pytorch/vision:v0.13.0', 'alexnet', pretrained=True)
modelA2.to(device)
modelB2 = torch.hub.load('pytorch/vision:v0.13.0', 'resnet50', pretrained=True)
modelB2.to(device)
modelC2 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
modelC2.aux_logits = False 
modelC2.to(device)
modelD2 = torch.hub.load('pytorch/vision:v0.13.0', 'vgg16', pretrained=True)
modelD2.to(device)
modelA2.classifier[6] = torch.nn.Linear(4096,11)
modelB2.fc = torch.nn.Linear(2048, 11)
modelC2.fc = torch.nn.Linear(2048, 11)
modelD2.classifier[6] = torch.nn.Linear(4096,11)
model3 = MyEnsemble2(modelA2,modelB2)
model3.to(device)
criterion3 = torch.nn.CrossEntropyLoss()
optimizer3 = torch.optim.SGD(model3.parameters(), lr=LEARNING_RATE)


if os.path.getsize(PATH1) > 0:
  checkpoint = torch.load(PATH1, map_location=torch.device(device))
  modelA2.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH2) > 0:
  checkpoint = torch.load(PATH2, map_location=torch.device(device))
  modelC2.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH3) > 0:
  checkpoint = torch.load(PATH3, map_location=torch.device(device))
  modelB2.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH4) > 0:
  checkpoint = torch.load(PATH4, map_location=torch.device(device))
  modelD2.load_state_dict(checkpoint['model_state_dict'])

epoch3 = 0
loss3 = 0.0
best_acc3 = 0.0
best_model_weights3 = []

if os.path.getsize(PATH11) > 0:
  checkpoint = torch.load(PATH11, map_location=torch.device(device))
  model3.load_state_dict(checkpoint['model_state_dict'])
  optimizer3.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch3 = checkpoint['epoch']
  loss3 = checkpoint['loss']
  best_acc3 = checkpoint['best_acc']
  best_model_weights3 = copy.deepcopy(model3.state_dict())


model3_info = {'train_loss' : [] , 'val_acc' : []}
PATH3_acc = "/content/drive/MyDrive/Test103/path3acc.pth"
if os.path.getsize(PATH3_acc) > 0:
  model3_info = torch.load(PATH3_acc, map_location=torch.device(device))



#Avg Ensemble
modelA3 = torch.hub.load('pytorch/vision:v0.13.0', 'alexnet', pretrained=True)
modelA3.to(device)
modelB3 = torch.hub.load('pytorch/vision:v0.13.0', 'resnet50', pretrained=True)
modelB3.to(device)
modelC3 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
modelC3.aux_logits = False 
modelC3.to(device)
modelD3 = torch.hub.load('pytorch/vision:v0.13.0', 'vgg16', pretrained=True)
modelD3.to(device)
modelA3.classifier[6] = torch.nn.Linear(4096,11)
modelB3.fc = torch.nn.Linear(2048, 11)
modelC3.fc = torch.nn.Linear(2048, 11)
modelD3.classifier[6] = torch.nn.Linear(4096,11)
model4 = MyEnsemble2(modelA3,modelB3)
model4.to(device)
criterion4 = torch.nn.CrossEntropyLoss()
optimizer4 = torch.optim.SGD(model4.parameters(), lr=LEARNING_RATE)


if os.path.getsize(PATH5) > 0:
  checkpoint = torch.load(PATH5, map_location=torch.device(device))
  modelA3.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH6) > 0:
  checkpoint = torch.load(PATH6, map_location=torch.device(device))
  modelC3.load_state_dict(checkpoint['model_state_dict'])

if os.path.getsize(PATH7) > 0:
  checkpoint = torch.load(PATH7, map_location=torch.device(device))
  modelB3.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH8) > 0:
  checkpoint = torch.load(PATH8, map_location=torch.device(device))
  modelD3.load_state_dict(checkpoint['model_state_dict'])

epoch4 = 0
loss4 = 0.0
best_acc4 = 0.0
best_model_weights4 = []

if os.path.getsize(PATH12) > 0:
  checkpoint = torch.load(PATH12, map_location=torch.device(device))
  model4.load_state_dict(checkpoint['model_state_dict'])
  optimizer4.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch4 = checkpoint['epoch']
  loss4 = checkpoint['loss']
  best_acc4 = checkpoint['best_acc']
  best_model_weights4 = copy.deepcopy(model4.state_dict())


model4_info = {'train_loss' : [] , 'val_acc' : []}
PATH4_acc = "/content/drive/MyDrive/Test103/path4acc.pth"
if os.path.getsize(PATH4_acc) > 0:
  model4_info = torch.load(PATH4_acc, map_location=torch.device(device))


#Base Ensemble Avg
modelA4 = torch.hub.load('pytorch/vision:v0.13.0', 'alexnet', pretrained=True)
modelA4.to(device)
modelC4 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
modelC4.aux_logits = False 
modelC4.to(device)
modelB4 = torch.hub.load('pytorch/vision:v0.13.0', 'resnet50', pretrained=True)
modelB4.to(device)
modelD4 = torch.hub.load('pytorch/vision:v0.13.0', 'vgg16', pretrained=True)
modelD4.to(device)
modelA4.classifier[6] = torch.nn.Linear(4096,11)
modelB4.fc = torch.nn.Linear(2048, 11)
modelC4.fc = torch.nn.Linear(2048, 11)
modelD4.classifier[6] = torch.nn.Linear(4096,11)
model5 = MyEnsemble2(modelA4,modelB4)
model5.to(device)
criterion5 = torch.nn.CrossEntropyLoss()
optimizer5 = torch.optim.SGD(model5.parameters(), lr=LEARNING_RATE)
if os.path.getsize(PATH1) > 0:
  checkpoint = torch.load(PATH1, map_location=torch.device(device))
  modelA4.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH2) > 0:
  checkpoint = torch.load(PATH2, map_location=torch.device(device))
  modelC4.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH3) > 0:
  checkpoint = torch.load(PATH3, map_location=torch.device(device))
  modelB4.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH4) > 0:
  checkpoint = torch.load(PATH4, map_location=torch.device(device))
  modelD4.load_state_dict(checkpoint['model_state_dict'])

epoch5 = 0
loss5 = 0.0
best_acc5 = 0.0
best_model_weights5 = []
if os.path.getsize(PATH13) > 0:
  checkpoint = torch.load(PATH13, map_location=torch.device(device))
  model5.load_state_dict(checkpoint['model_state_dict'])
  optimizer5.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch5 = checkpoint['epoch']
  loss5 = checkpoint['loss']
  best_acc5 = checkpoint['best_acc']
  best_model_weights5 = copy.deepcopy(model5.state_dict())


model5_info = {'train_loss' : [] , 'val_acc' : []}
PATH5_acc = "/content/drive/MyDrive/Test103/path5acc.pth"
if os.path.getsize(PATH5_acc) > 0:
  model5_info = torch.load(PATH5_acc, map_location=torch.device(device))

AVG_PATH1 = "/content/drive/MyDrive/Test103/avgpath1.pth"
avg_model_weights1 = []
if os.path.getsize(AVG_PATH1) > 0:
  checkpoint = torch.load(AVG_PATH1)
  avg_model_weights1 = checkpoint['model_state_dict']

prev_model1 = []
if len(avg_model_weights1) != 0:
  prev_model1 = avg_model_weights1


#Avg Ensemble Avg
modelA5 = torch.hub.load('pytorch/vision:v0.13.0', 'alexnet', pretrained=True)
modelA5.to(device)
modelC5 = torch.hub.load('pytorch/vision:v0.13.0', 'inception_v3', pretrained=True)
modelC5.aux_logits = False 
modelC5.to(device)
modelB5 = torch.hub.load('pytorch/vision:v0.13.0', 'resnet50', pretrained=True)
modelB5.to(device)
modelD5 = torch.hub.load('pytorch/vision:v0.13.0', 'vgg16', pretrained=True)
modelD5.to(device)
modelA5.classifier[6] = torch.nn.Linear(4096,11)
modelB5.fc = torch.nn.Linear(2048, 11)
modelC5.fc = torch.nn.Linear(2048, 11)
modelD5.classifier[6] = torch.nn.Linear(4096,11)
model6 = MyEnsemble2(modelA5,modelB5)
model6.to(device)
criterion6 = torch.nn.CrossEntropyLoss()
optimizer6 = torch.optim.SGD(model6.parameters(), lr=LEARNING_RATE)
if os.path.getsize(PATH5) > 0:
  checkpoint = torch.load(PATH5, map_location=torch.device(device))
  modelA5.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH6) > 0:
  checkpoint = torch.load(PATH6, map_location=torch.device(device))
  modelC5.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH7) > 0:
  checkpoint = torch.load(PATH7, map_location=torch.device(device))
  modelB5.load_state_dict(checkpoint['model_state_dict'])
if os.path.getsize(PATH8) > 0:
  checkpoint = torch.load(PATH8, map_location=torch.device(device))
  modelD5.load_state_dict(checkpoint['model_state_dict'])

epoch6 = 0
loss6 = 0.0
best_acc6 = 0.0
best_model_weights6 = []

if os.path.getsize(PATH14) > 0:
  checkpoint = torch.load(PATH14, map_location=torch.device(device))
  model6.load_state_dict(checkpoint['model_state_dict'])
  optimizer6.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch6 = checkpoint['epoch']
  loss6 = checkpoint['loss']
  best_acc6 = checkpoint['best_acc']
  best_model_weights6 = copy.deepcopy(model6.state_dict())


model6_info = {'train_loss' : [] , 'val_acc' : []}
PATH6_acc = "/content/drive/MyDrive/Test103/path6acc.pth"
if os.path.getsize(PATH6_acc) > 0:
  model6_info = torch.load(PATH6_acc, map_location=torch.device(device))

AVG_PATH2 = "/content/drive/MyDrive/Test103/avgpath2.pth"
avg_model_weights2 = []
if os.path.getsize(AVG_PATH2) > 0:
  checkpoint = torch.load(AVG_PATH2)
  avg_model_weights2 = checkpoint['model_state_dict']

prev_model2 = []
if len(avg_model_weights2) != 0:
  prev_model2 = avg_model_weights2

start_time = time.time()
new_time1 = time.time()
total_step = len(trainset_loader1)



In [ ]:
for epoch in range(NUM_EPOCHS):

#NOW NODEL 1

    i = 0
    new_time1 = time.time()
    start_time = time.time()
    model1.train()
    model2.train()
    model3.train()
    model4.train()
    model5.train()
    model6.train()
    print('Epoch [{}/{}] --------- '.format(epoch+1, NUM_EPOCHS))
    for a,b in zip(trainset_loader1,trainset_loader2):
        optimizer1.zero_grad()
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs1 = model1(images,images)
        loss1 = criterion1(outputs1, labels)
        loss1.backward()
        optimizer1.step()
    model1_info['train_loss'].append(loss1.item())

    print ('Model 1  Step [{}/{}], Loss: {:.4f}' 
                   .format(i+1, total_step, loss1.item()))
  
    # Validation

    model1.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for a,b in zip(validset_loader1,validset_loader2):
            images,labels = a
            v3_images,v3_labels = b
            images = images.to(device)
            labels = labels.to(device)
            v3_images = v3_images.to(device)
            outputs =  model1(images,images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, v3_images, v3_labels, outputs
    
        print('Accuracy of the network on the {} valid set images: {} %'.format(total, 100 * correct / total)) 
        model1_info['val_acc'].append((100 * correct / total))
        torch.save(model1_info,PATH1_acc)
        if (100 * correct / total) > best_acc1 : 
            best_acc1 = (100 * correct / total)
            best_model_weights1 = copy.deepcopy(model1.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model1.state_dict(),
            'optimizer_state_dict': optimizer1.state_dict(),
            'loss': loss1,
            'best_acc' : best_acc1,
            },PATH9)

    for a,b in zip(trainset_loader1,trainset_loader2):
        optimizer1.zero_grad()
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs1 = model2(images,images)
        loss2 = criterion2(outputs1, labels)
        loss2.backward()
        optimizer2.step()

    model2_info['train_loss'].append(loss2.item())
    print ('Model 2  Step [{}/{}], Loss: {:.4f}' 
                   .format(i+1, total_step, loss2.item()))
  
    # Validation

    model2.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for a,b in zip(validset_loader1,validset_loader2):
            images,labels = a
            v3_images,v3_labels = b
            images = images.to(device)
            labels = labels.to(device)
            v3_images = v3_images.to(device)
            outputs =  model2(images,images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, v3_images, v3_labels, outputs
    
        print('Accuracy of the network on the {} valid set images: {} %'.format(total, 100 * correct / total)) 
        model2_info['val_acc'].append((100 * correct / total))
        torch.save(model2_info,PATH2_acc)
        if (100 * correct / total) > best_acc2 : 
            best_acc2 = (100 * correct / total)
            best_model_weights2 = copy.deepcopy(model2.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model2.state_dict(),
            'optimizer_state_dict': optimizer2.state_dict(),
            'loss': loss2,
            'best_acc' : best_acc2,
            },PATH10)

    for a,b in zip(trainset_loader1,trainset_loader2):
        optimizer1.zero_grad()
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs1 = model3(images,images)
        loss3 = criterion3(outputs1, labels)
        loss3.backward()
        optimizer3.step()

    model3_info['train_loss'].append(loss3.item())
    print ('Model 3  Step [{}/{}], Loss: {:.4f}' 
                   .format(i+1, total_step, loss3.item()))
  
    # Validation

    model3.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for a,b in zip(validset_loader1,validset_loader2):
            images,labels = a
            v3_images,v3_labels = b
            images = images.to(device)
            labels = labels.to(device)
            v3_images = v3_images.to(device)
            outputs =  model3(images,images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, v3_images, v3_labels, outputs
    
        print('Accuracy of the network on the {} valid set images: {} %'.format(total, 100 * correct / total)) 
        model3_info['val_acc'].append((100 * correct / total))
        torch.save(model3_info,PATH3_acc)
        if (100 * correct / total) > best_acc3 : 
            best_acc3 = (100 * correct / total)
            best_model_weights3 = copy.deepcopy(model3.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model3.state_dict(),
            'optimizer_state_dict': optimizer3.state_dict(),
            'loss': loss3,
            'best_acc' : best_acc3,
            },PATH11)

    for a,b in zip(trainset_loader1,trainset_loader2):
        optimizer1.zero_grad()
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs1 = model4(images,images)
        loss4 = criterion4(outputs1, labels)
        loss4.backward()
        optimizer4.step()

    model4_info['train_loss'].append(loss4.item())
    print ('Model 4  Step [{}/{}], Loss: {:.4f}' 
                    .format(i+1, total_step, loss4.item()))

    # Validation

    model4.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for a,b in zip(validset_loader1,validset_loader2):
            images,labels = a
            v3_images,v3_labels = b
            images = images.to(device)
            labels = labels.to(device)
            v3_images = v3_images.to(device)
            outputs =  model4(images,images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, v3_images, v3_labels, outputs

        print('Accuracy of the network on the {} valid set images: {} %'.format(total, 100 * correct / total)) 
        model4_info['val_acc'].append((100 * correct / total))
        torch.save(model4_info,PATH4_acc)
        if (100 * correct / total) > best_acc4 : 
            best_acc4 = (100 * correct / total)
            best_model_weights4 = copy.deepcopy(model4.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model4.state_dict(),
            'optimizer_state_dict': optimizer4.state_dict(),
            'loss': loss4,
            'best_acc' : best_acc4,
            },PATH12)



    for a,b in zip(trainset_loader1,trainset_loader2):
        optimizer1.zero_grad()
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs1 = model5(images,images)
        loss5 = criterion5(outputs1, labels)
        loss5.backward()
        optimizer5.step()

    model5_info['train_loss'].append(loss5.item())
    print ('Model 5  Step [{}/{}], Loss: {:.5f}' 
                    .format(i+1, total_step, loss5.item()))

    if epoch+1 == 5:

        prev_model1 = best_model_weights5

        
    elif epoch+1 > 5:

        temp_model = copy.deepcopy(model5.state_dict())
        for key in prev_model1:
            prev_model1[key] = (prev_model1[key] + temp_model[key]) / 2.0
        model5.load_state_dict(prev_model1)
        torch.save({
                'model_state_dict': model5.state_dict(),
                },AVG_PATH1) 

    # Validation

    model5.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for a,b in zip(validset_loader1,validset_loader2):
            images,labels = a
            v3_images,v3_labels = b
            images = images.to(device)
            labels = labels.to(device)
            v3_images = v3_images.to(device)
            outputs =  model5(images,images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, v3_images, v3_labels, outputs

        print('Accuracy of the network on the {} valid set images: {} %'.format(total, 100 * correct / total)) 
        model5_info['val_acc'].append((100 * correct / total))
        torch.save(model5_info,PATH5_acc)
        if (100 * correct / total) > best_acc5 : 
            best_acc5 = (100 * correct / total)
            best_model_weights5 = copy.deepcopy(model5.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model5.state_dict(),
            'optimizer_state_dict': optimizer5.state_dict(),
            'loss': loss5,
            'best_acc' : best_acc5,
            },PATH13)

    for a,b in zip(trainset_loader1,trainset_loader2):
        optimizer1.zero_grad()
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs1 = model6(images,images)
        loss6 = criterion6(outputs1, labels)
        loss6.backward()
        optimizer6.step()

    model6_info['train_loss'].append(loss6.item())
    print ('Model 6 Step [{}/{}], Loss: {:.5f}' 
                    .format(i+1, total_step, loss6.item()))

    if epoch+1 == 5:

        prev_model2 = best_model_weights6

        
    elif epoch+1 > 5:

        temp_model = copy.deepcopy(model6.state_dict())
        for key in prev_model2:
            prev_model2[key] = (prev_model2[key] + temp_model[key]) / 2.0
        model6.load_state_dict(prev_model2)
        torch.save({
                'model_state_dict': model6.state_dict(),
                },AVG_PATH2) 

    # Validation

    model6.eval()
    with torch.set_grad_enabled(False):
        correct = 0
        total = 0
        for a,b in zip(validset_loader1,validset_loader2):
            images,labels = a
            v3_images,v3_labels = b
            images = images.to(device)
            labels = labels.to(device)
            v3_images = v3_images.to(device)
            outputs =  model6(images,images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, v3_images, v3_labels, outputs

        print('Accuracy of the network on the {} valid set images: {} %'.format(total, 100 * correct / total)) 
        model6_info['val_acc'].append((100 * correct / total))
        torch.save(model6_info,PATH6_acc)
        if (100 * correct / total) > best_acc6 : 
            best_acc6 = (100 * correct / total)
            best_model_weights6 = copy.deepcopy(model6.state_dict())
            torch.save({
            'epoch': epoch,
            'model_state_dict': model6.state_dict(),
            'optimizer_state_dict': optimizer6.state_dict(),
            'loss': loss6,
            'best_acc' : best_acc6,
            },PATH14)





model1.load_state_dict(best_model_weights1)
model1.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(validset_loader1,validset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model1(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 1 on the {} validation images: {} %'.format(total, 100 * correct / total)) 

model1.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(evaluationset_loader1,evaluationset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model1(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 1 on the {} evaluation images: {} %'.format(total, 100 * correct / total))

model2.load_state_dict(best_model_weights2)
model2.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(validset_loader1,validset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model2(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 2 on the {} validation images: {} %'.format(total, 100 * correct / total)) 

model2.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(evaluationset_loader1,evaluationset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model2(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 2 on the {} evaluation images: {} %'.format(total, 100 * correct / total))

model3.load_state_dict(best_model_weights3)
model3.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(validset_loader1,validset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model3(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 3 on the {} validation images: {} %'.format(total, 100 * correct / total)) 

model3.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(evaluationset_loader1,evaluationset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model3(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 3 on the {} evaluation images: {} %'.format(total, 100 * correct / total))

model4.load_state_dict(best_model_weights4)
model4.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(validset_loader1,validset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model4(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 4 on the {} validation images: {} %'.format(total, 100 * correct / total)) 

model4.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(evaluationset_loader1,evaluationset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model4(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 4 on the {} evaluation images: {} %'.format(total, 100 * correct / total))

model5.load_state_dict(best_model_weights5)
model5.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(validset_loader1,validset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model5(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 5 on the {} validation images: {} %'.format(total, 100 * correct / total)) 

model5.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(evaluationset_loader1,evaluationset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model5(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 5 on the {} evaluation images: {} %'.format(total, 100 * correct / total))

model6.load_state_dict(best_model_weights6)
model6.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(validset_loader1,validset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model6(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 6 on the {} validation images: {} %'.format(total, 100 * correct / total)) 

model6.eval()
with torch.set_grad_enabled(False):
    correct = 0
    total = 0
    for a,b in zip(evaluationset_loader1,evaluationset_loader2):
        images,labels = a
        v3_images,v3_labels = b
        images = images.to(device)
        labels = labels.to(device)
        v3_images = v3_images.to(device)
        outputs =  model6(images,images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, v3_images, v3_labels, outputs

    print('Accuracy of the model 6 on the {} evaluation images: {} %'.format(total, 100 * correct / total))
plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 1")
plt.plot(model1_info['val_acc'],label="val")
plt.plot(model1_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 2")
plt.plot(model2_info['val_acc'],label="val")
plt.plot(model2_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 3")
plt.plot(model3_info['val_acc'],label="val")
plt.plot(model3_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()


plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 4")
plt.plot(model4_info['val_acc'],label="val")
plt.plot(model4_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 4")
plt.plot(model5_info['val_acc'],label="val")
plt.plot(model5_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()


plt.figure(figsize=(10,5))
plt.title("Training loss and Validation Accuracy of model 4")
plt.plot(model6_info['val_acc'],label="val")
plt.plot(model6_info['train_loss'],label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()